In [1]:
from ml_fingerprint import ml_fingerprint, example_models, remote

In [2]:
from datetime import datetime
d = datetime.now()

In [3]:
model, score = example_models.rain_classifier()

In [4]:
scores = {'Accuracy': score}
description = '''Linear classifier that predicts if it will rain tomorrow.
    The dataset is taken from Kaggle and can be found here:
    https://www.kaggle.com/jsphyg/weather-dataset-rattle-package
    It consists of daily weather data in australian cities, and whether
    it rained the next day or not.'''

In [5]:
server = remote.RemoteServer('http://localhost:5000')

In [8]:
server.insert_model(model, 'australia_rain_predict', True, 'Classification', scores, '1.0.0', {}, datetime.now(), description, "Jaime Solsona")

In [9]:
model = example_models.vanderplas_regression()

description = '''Example linear regressor taken from "Python Data Science Handbook", by Jake VanderPlas.
    The training points follow the function y = -2x + z, with an added noise.
    The regressor then tries to predict the value of y based on x and z values.'''

In [22]:
import numpy as np
rng = np.random.RandomState(1)
X = rng.randn(100, 2)
y = -2*X[:,0] + X[:,1]
y_pred = model.predict(X)



In [25]:
from sklearn import metrics
mse = metrics.mean_squared_error(y, y_pred)
r2 = metrics.r2_score(y, y_pred)
scores = {}
scores['MSE'] = mse
scores['R2'] = r2

In [27]:
server.insert_model(model, 'vanderplas_regression_example', True, 'Regression', scores, '1.0.0', {}, datetime.now(), description, "Jaime Solsona")